<a href="https://colab.research.google.com/github/utkarshminhas/violence-detection-fyp/blob/main/notebooks/working_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from tqdm import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf
import random
from glob import glob
from sklearn.utils import shuffle
import numpy as np

In [ ]:
processed_dir = '/content/drive/Shareddrives/Datasets/Raw RWF-2000 (without_mog)'
print(len(os.listdir(processed_dir)))
print(len(os.listdir(processed_dir+"/Fight")))
print(len(os.listdir(processed_dir+'/NonFight')))

# image_size = (299, 299)
image_size = (128, 128)
nonFight, fight = (np.array(0), np.array(1))
# nonFight, fight = ((None,0), (None,1))


2
1000
1000


In [ ]:
def load_initial_data():
    x = []
    y = []

    # Fights
    allFights = sorted(glob(os.path.join(processed_dir, 'Fight/*')))
    x.extend(allFights)
    y.extend([fight] * len(allFights))

    # Non Fights
    allNonFights = sorted(glob(os.path.join(processed_dir, 'NonFight/*')))
    x.extend(allNonFights)
    y.extend([nonFight] * len(allNonFights))

    return x, y

In [ ]:
videoPaths, labels = load_initial_data()
df = pd.DataFrame(list(zip(videoPaths, labels)), 
               columns =['Name', 'Label']) 
df=shuffle(df)
# df

In [ ]:
allvideos=df.to_numpy()
videos,labels=[],[]
for video in allvideos:
  videos.append(video[0])
  labels.append(video[1])


# train_videoPaths,train_labels=videos[:1500],labels[:1500]
# val_videoPaths,val_labels=videos[1500:1900],labels[1500:1900]

In [ ]:
def get_image(imagePath):
    img = cv2.imread(imagePath, cv2.IMREAD_COLOR) # Read in RGB  directly
    img = cv2.resize(img, image_size)
    img = img / 255.0
    img = img.astype(np.float32)
    return img

In [ ]:
counter=0
def preprocess(videoPath, label):
    def myFunction(videoPath):
        global counter
        videoPath = videoPath.decode()
        frames = []
        for frame_path in glob(os.path.join(videoPath, '*')):
            image = get_image(frame_path)
            frames.append(image)
        counter+=1
        if counter%50==0:
            print(counter)
        else:
            print(counter,end="->")
        return np.asarray(frames)
    video = tf.numpy_function(myFunction, [videoPath], [tf.float32])[0]
    # label=tf.convert_to_tensor(label)
    # print(video.shape)
    # label=tf.one_hot(label[0],2)
    return video, label

In [ ]:
def tf_dataset(videoPaths, labels, batch_size=5):
    dataset = tf.data.Dataset.from_tensor_slices((videoPaths, labels))
    dataset = dataset.map(preprocess,num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.cache()
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

In [ ]:
ds=tf_dataset(videos[:10], labels[:10], batch_size=5)
val=tf_dataset(videos[10:15], labels[10:15], batch_size=5)
batched_train_dataset_len=len(ds)
train_steps=batched_train_dataset_len

batched_val_dataset_len=len(val)
val_steps=batched_val_dataset_len

print(train_steps,val_steps)


2 1


In [ ]:
from tensorflow.keras import layers
import numpy as np
from keras.layers import TimeDistributed, Conv2D, Dense, MaxPooling2D, \
Flatten, LSTM, Dropout, BatchNormalization
from keras import models
import itertools

import keras
from keras import optimizers
# import tensorflow_addons as tfa

import datetime

In [ ]:
input_shape = (30, image_size[0], image_size[1], 3)
num_epochs = 5
print(input_shape)

(30, 128, 128, 3)


In [ ]:
strategy= tf.distribute.MultiWorkerMirroredStrategy()

with strategy.scope():
    model = models.Sequential(
        [
            TimeDistributed(
                Conv2D(32, (3, 3), activation=tf.nn.relu), 
                input_shape=input_shape
            ),
            TimeDistributed(MaxPooling2D((2, 2), strides=(1, 1))),
            TimeDistributed(Conv2D(32, (4, 4), activation=tf.nn.relu)),
            TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))),
            TimeDistributed(Conv2D(64, (4, 4), activation=tf.nn.relu)),
            TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))),
            TimeDistributed(Flatten()),
            # Dropout(0.6),
            LSTM(256, return_sequences=False, dropout=0.5,),
            Dropout(0.7),
            Flatten(),
            Dense(64, activation=tf.nn.relu),
            Dense(32, activation=tf.nn.relu),
            Dense(1, activation=tf.nn.sigmoid)
        ]
    )

    model.summary()

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
log_dir = "/content/drive/Shareddrives/Datasets/Tensorboard logs" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,)


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
optimizer = optimizers.Adam()

model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=['accuracy']
)

In [ ]:

with tf.device('/device:GPU:0'):
    history = model.fit(
        ds,
        validation_data=val,
        epochs=num_epochs,
        steps_per_epoch=train_steps,
        validation_steps=val_steps,
        # batch_size=1,
        # validation_batch_size=1,
        verbose=2,
        use_multiprocessing=True,
        workers=32,

        callbacks=[tensorboard_callback]
        
        
    )

Epoch 1/5
1->2->3->4->5->6->7->8->9->10->11->12->13->14->15->2/2 - 144s - loss: 0.7178 - accuracy: 0.5000 - val_loss: 0.7915 - val_accuracy: 0.6000
Epoch 2/5
2/2 - 73s - loss: 0.5742 - accuracy: 0.7000 - val_loss: 0.9376 - val_accuracy: 0.6000
Epoch 3/5
2/2 - 73s - loss: 0.5461 - accuracy: 0.8000 - val_loss: 0.9486 - val_accuracy: 0.6000
Epoch 4/5
2/2 - 73s - loss: 0.5727 - accuracy: 0.9000 - val_loss: 1.0343 - val_accuracy: 0.6000
Epoch 5/5
2/2 - 83s - loss: 0.8858 - accuracy: 0.7000 - val_loss: 1.0491 - val_accuracy: 0.6000


In [ ]:
%tensorboard --logdir '/content/drive/Shareddrives/Datasets/Tensorboard logs20210225-100754'

UsageError: Line magic function `%tensorboard` not found.


In [ ]:
import datetime
model.save('/content/drive/Shareddrives/Final Year Project/Saved Model/' + '24Feb_2020_tfds_iter1.h5',save_format='h5')